In [ ]:
# Function to get the numberic value from the LLM Response

import re

pattern = r"^(?:[1-9](?:\.[0-9])?|10)(?!\d)"

def get_llm_response(llm_response, prompt_response = False):
  try:
    if not prompt_response:
      return llm_response[0]["generated_text"][1]["content"].strip()
    else:
      # try to cast as float as is
      try:
        return float(llm_response[0]["generated_text"][1]["content"].strip())
      except Exception:
        try:
          match_ = re.match(pattern, llm_response[0]["generated_text"][1]["content"].strip())
          if match_:
            value = float(match_.group())
            return value
          else:
            return None
        except Exception:
          return None

  except Exception as e:
    print(e)
    return None


In [ ]:

# Generate the initial pipeline
from transformers import pipeline

pipe = pipeline("text-generation", model="microsoft/Phi-3-mini-4k-instruct", trust_remote_code=False)
messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe(messages)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

Device set to use cuda:0


[{'generated_text': [{'role': 'user', 'content': 'Who are you?'},
   {'role': 'assistant',
    'content': " I am Phi, Microsoft's language model. How can I assist you?"}]}]

In [ ]:
# Load the all student_json

import json

student_object = json.load(open("sample_data/all_per_student.json"))

In [ ]:
import pickle

# MASTER is the global dictionary that contains all the data
# Initially set to {} but saved as a pickle and reloaded to continue adding data
with open("sample_data/ontology.pickle", "rb") as f:
  MASTER = pickle.load(f)

In [ ]:
import re
import ast
pattern = r"\{[a-zA-Z_][a-zA-Z0-9_]*\}"


def get_final_prompt(prompt_info, noun, student):
  '''
  This custom function will attach a noun to the end of the prompt to handle most cases
  while gathering the actual prompt and property
  '''

  if(type(prompt_info) != dict):
    return (None, None, None)
  try:

    metakeys = list(prompt_info.keys())
    # Most students have pre-prompt, but some have prompt
    # Comment the correct one for customized student functions
    if 'pre-prompt' not in metakeys:
      return (None, None, None)
    if 'prompt' not in metakeys:
      return (None, None, None)
    elif 'property' not in metakeys:
      return (None, None, None)
    else:
      prompt_val = prompt_info['pre-prompt']
      #prompt_val = prompt_info['prompt']
      propert_val = prompt_info['property']
      final_prompt = prompt_val + " " + noun
      return (prompt_val, propert_val, final_prompt)
  except Exception as e:
    print("PROMPT ISSUE")
    print(e)
    return (None, None, None)







In [ ]:
import copy
import pickle

def run_ontology_generation(run_number):

  print("RUNNING LOOP {}".format(run_number))
  _shell = {"categories": set([]), "properties": set([]), "metdata": {}}
  _meta_shell = {'values': [], 'min': '', 'max': '', 'mean': '', 'std': ''}

  # SKIP THE FOLLOWING STUDENTS.  # THOSE WILL

  SKIP_STUDENT = ['347003_','436515_', '61264_', '122358_', '393174_', '389880_', '440231_']
  EXECUTE = ["394991_"]

  for student in student_object:

    if student in SKIP_STUDENT:
      continue
    # Specialized Case to run this code on a specific student (typically if they have a slight variation)
    # if student not in EXECUTE:
    #   continue
    prompts = student_object[student]['prompts']
    if len(prompts) == 0:
      continue

    print(student)

    for prompt in prompts:
      # Loop through each prompt

      for noun in student_object[student]['nouns']:
        # Loop Through each noun,

        # Get the prompt to send and property value back
        prompt_val, property_val, prompt_to_send = get_final_prompt(prompt, noun, student)
        if prompt_val is None:
          continue

        # Send prompt
        messages = [
        {"role": "user", "content": prompt_to_send},
        ]
        response = pipe(messages)
        # Get the LLM result parsed
        response_parsed = get_llm_response(response, prompt_response=True)
        if response_parsed:
          try:
            result = response_parsed
            MASTER[noun]["properties"].add(property_val)
            if property_val not in MASTER[noun]["metdata"]:
              MASTER[noun]["metdata"][property_val] = copy.deepcopy(_meta_shell)
              MASTER[noun]["metdata"][property_val]["values"].append(result)
            else:
              MASTER[noun]["metdata"][property_val]["values"].append(result)

          except Exception as e:
            print(e)
            continue

  with open("sample_data/ontology.pickle", "wb") as f:
    pickle.dump(MASTER, f)

  print("RUN {} COMPLETE".format(run_number))


In [ ]:
for i in range(5):
  run_ontology_generation(i)

RUNNING LOOP 0
394991_
RUN 0 COMPLETE
RUNNING LOOP 1
394991_


KeyboardInterrupt: 

THE REMAINING BLOCKS OF CODE ARE CUSTOM FUNCTIONS TO GET THE REMAINING STUDENTS' NOUNS ADDED TO THE ONTOLOGY (THAT WERE FEASIBLE)

In [ ]:
#### 347003_

def run_student_347003():
  _meta_shell = {'values': [], 'min': '', 'max': '', 'mean': '', 'std': ''}

  student = '347003_'
  prompts = student_object[student]['prompts']
  for prompt in prompts:
    prompt_val = prompt['pre-prompt']
    # Custom function since this student had all the prompts already derived
    # So need to extract it from the prompt (last word)
    noun = prompt_val.split(' ')[-1].replace('?','').strip().lower()
    property_val = prompt['property']
    messages = [
        {"role": "user", "content": prompt_val},
        ]
    response = pipe(messages)
    response_parsed = get_llm_response(response, prompt_response=True)
    if response_parsed:
      try:
        result = response_parsed
        MASTER[noun]["properties"].add(property_val)
        if property_val not in MASTER[noun]["metdata"]:
          MASTER[noun]["metdata"][property_val] = copy.deepcopy(_meta_shell)
          MASTER[noun]["metdata"][property_val]["values"].append(result)
        else:
          MASTER[noun]["metdata"][property_val]["values"].append(result)

      except Exception as e:
        print(e)
        continue


  with open("sample_data/master.pickle", "wb") as f:
    pickle.dump(MASTER, f)






In [ ]:
for i in range(5):
  print(i)
  run_student_347003()

0
1
2
3
4


In [ ]:
#436515_
def run_student_436515():
  _meta_shell = {'values': [], 'min': '', 'max': '', 'mean': '', 'std': ''}

  student = '436515_'
  prompts = student_object[student]['prompts']
  for prompt in prompts:
    prompt_val = prompt['pre-prompt']['content']
    property_val = prompt_val.split(":")[-1].strip() # Extract property
    #Needed to add custom logic because this student was not consistent in some cases
    #for their noun in the prompt and the noun in teh list
    noun = prompt['company'].lower().strip()
    if noun == "alphabet (google)":
      noun = "alphabet"
    if noun == "block (formerly square)":
      noun = "block"

    messages = [
        {"role": "user", "content": prompt_val},
        ]
    response = pipe(messages)
    response_parsed = get_llm_response(response, prompt_response=True)
    if response_parsed:
      try:
        result = response_parsed
        MASTER[noun]["properties"].add(property_val)
        if property_val not in MASTER[noun]["metdata"]:
          MASTER[noun]["metdata"][property_val] = copy.deepcopy(_meta_shell)
          MASTER[noun]["metdata"][property_val]["values"].append(result)
        else:
          MASTER[noun]["metdata"][property_val]["values"].append(result)

      except Exception as e:
        print(e)
        continue


  with open("sample_data/master.pickle", "wb") as f:
    pickle.dump(MASTER, f)

In [ ]:
run_student_436515()

KeyboardInterrupt: 

In [ ]:
#61264_
def run_student_61264():
  _meta_shell = {'values': [], 'min': '', 'max': '', 'mean': '', 'std': ''}

  student = '61264_'
  prompts = student_object[student]['prompts']
  for prompt in prompts:
    prompt_val = prompt['full_prompt']
    noun = prompt['noun'].lower()
    property_val = prompt['property']

    messages = [
        {"role": "user", "content": prompt_val},
        ]
    response = pipe(messages)
    response_parsed = get_llm_response(response, prompt_response=True)
    if response_parsed:
      try:
        result = response_parsed
        MASTER[noun]["properties"].add(property_val)
        if property_val not in MASTER[noun]["metdata"]:
          MASTER[noun]["metdata"][property_val] = copy.deepcopy(_meta_shell)
          MASTER[noun]["metdata"][property_val]["values"].append(result)
        else:
          MASTER[noun]["metdata"][property_val]["values"].append(result)

      except Exception as e:
        print(e)
        continue


  with open("sample_data/master.pickle", "wb") as f:
    pickle.dump(MASTER, f)



In [ ]:
for i in range(5):
  print(i)
  run_student_61264()

0
1
2
3
4


In [ ]:
#122358_
import re


def run_student_122358():
  _meta_shell = {'values': [], 'min': '', 'max': '', 'mean': '', 'std': ''}

  student = '122358_'
  prompts = student_object[student]['prompts']
  for prompt in prompts:
    prompt_val = prompt
    # Had to use RE in the prompt to get the noun
    noun = re.search(r'Noun:\s*(\w+)', prompt_val)
    if noun:
      noun = noun.group(1).strip()
    else:
      print('unfound noun in {}'.format(prompt_val))
      continue
    # Had to use RE in the prompt to get the property
    property_val = re.search(r'Property:\s*(\w+)', prompt_val)
    if property_val:
      property_val = property_val.group(1).strip()
    else:
      print('unfound property in {}'.format(prompt_val))
      continue

    messages = [
        {"role": "user", "content": prompt_val},
        ]
    response = pipe(messages)
    response_parsed = get_llm_response(response, prompt_response=True)
    if response_parsed:
      try:
        result = response_parsed
        MASTER[noun]["properties"].add(property_val)
        if property_val not in MASTER[noun]["metdata"]:
          MASTER[noun]["metdata"][property_val] = copy.deepcopy(_meta_shell)
          MASTER[noun]["metdata"][property_val]["values"].append(result)
        else:
          MASTER[noun]["metdata"][property_val]["values"].append(result)

      except Exception as e:
        print(e)
        continue


  with open("sample_data/master.pickle", "wb") as f:
    pickle.dump(MASTER, f)



In [ ]:
for i in range(5):
  print(i)
  run_student_122358()

0
1
2
3
4


In [ ]:
#393174_
def run_student_393174():
  _meta_shell = {'values': [], 'min': '', 'max': '', 'mean': '', 'std': ''}

  student = '393174_'
  prompts = student_object[student]['prompts']
  for prompt in prompts:
    prompt_val = prompt['full_prompt']
    noun = prompt['noun'].lower()
    property_val = prompt['property'].lower()

    messages = [
        {"role": "user", "content": prompt_val},
        ]
    response = pipe(messages)
    response_parsed = get_llm_response(response, prompt_response=True)


    if response_parsed:
      try:
        result = response_parsed
        MASTER[noun]["properties"].add(property_val)
        if property_val not in MASTER[noun]["metdata"]:
          MASTER[noun]["metdata"][property_val] = copy.deepcopy(_meta_shell)
          MASTER[noun]["metdata"][property_val]["values"].append(result)
        else:
          MASTER[noun]["metdata"][property_val]["values"].append(result)

      except Exception as e:
        print(e)
        continue


  with open("sample_data/ontology.pickle", "wb") as f:
    pickle.dump(MASTER, f)



In [ ]:
for i in range(5):
  print(i)
  run_student_393174()

0
1
2
3
4


In [ ]:
#440231_
def run_student_440231():
  _meta_shell = {'values': [], 'min': '', 'max': '', 'mean': '', 'std': ''}

  student = '440231_'

  nouns = student_object[student]['nouns']
  properties = student_object[student]['properties']
  for property_val in properties:
    for noun in nouns:
      _template = "Output ONLY a number from 1-10. Cannot use any other output aside from a 1-10 number. If Property is Weight, scale from 1 (weighs very light like a mouse) to 10 (weight is very heavy like an Elephant. Elephant must be a 10 for weight. Elephant can NEVER be a 1). If Property is Height, scale from 1 (very small like a Meerkat) to 10 (very tall like a giraffe. Giraffe is never 1). If Property is Speed, scale from 1 (very slow like a Sloth) to 10 (very fast like a Cheetah. Cheetah is never 1). If Property is Strength, scale from 1 (very weak like a Parrot) to 10 (very strong like an Elephant. Elephant is never 1). If Property is Intelligence, scale from 1 (very dumb like a Sloth) to 10 (very smart like an Orangutan. Orangutan is never 1). How would you rate this noun in terms of "
      prompt_val = _template.replace('this noun', 'a {}'.format(noun)) +" the property "+ property_val + "?"
      messages = [
          {"role": "user", "content": prompt_val},
          ]
      response = pipe(messages)
      response_parsed = get_llm_response(response, prompt_response=True)

      if response_parsed:
        try:
          result = response_parsed
          MASTER[noun]["properties"].add(property_val)
          if property_val not in MASTER[noun]["metdata"]:
            MASTER[noun]["metdata"][property_val] = copy.deepcopy(_meta_shell)
            MASTER[noun]["metdata"][property_val]["values"].append(result)
          else:
            MASTER[noun]["metdata"][property_val]["values"].append(result)

        except Exception as e:
          print(e)
          continue


  with open("sample_data/ontology.pickle", "wb") as f:
    pickle.dump(MASTER, f)

In [ ]:
for i in range(5):
  print(i)
  run_student_440231()


0
1
2
3
4


In [ ]:
# THIS WILL TAKE THE MASTER DICTIONARY AND ITERATE AND CALCULATE STATISTICS BASED OFF EACH NOUNS PROPERTY

import math
import statistics

count = 0
for noun, data in MASTER.items():
    if len(data['properties']) > 0:
        for property_ in data['properties']:
            metadata = data['metdata'][property_]
            metadata_values = metadata['values']
            if len(metadata_values) > 0:
                min_ = min(metadata_values)
                max_ = max(metadata_values)
                average = sum(metadata_values) / len(metadata_values)
                standard_deviation = math.sqrt((max_ - min_) / len(metadata_values))
                metadata['min'] = min_
                metadata['max'] = max_
                metadata['mean'] = average
                metadata['std'] = standard_deviation
                metadata['median'] = statistics.median(metadata_values)
                metadata['mode'] = statistics.mode(metadata_values)
    else:
        count+=1



with open('sample_data/ontology.pickle', 'wb') as f:
    pickle.dump(MASTER, f)

print("Number of Nouns without METADATA", count)

NameError: name 'MASTER' is not defined

Now i am going to generate the data for those students that didnt have prompts


In [ ]:
import copy
#"387331_"
def run_student_387331():
  _meta_shell = {'values': [], 'min': '', 'max': '', 'mean': '', 'std': ''}

  student = '387331_'

  nouns = student_object[student]['nouns']
  properties = student_object[student]['properties']
  for noun in nouns:
    for property_val in properties:
      _template = '''
      You are a semantic classifier of exercises and are tasked to rate an exercise in terms of a particular property in a range from 1-10 only.
      You must respond with a numeric rating ONLY in range from 1-10 based on the propery.  Only respond with a number and nothing else.
      1. If the property is cardiovascular benefit, a rating of 1 means the exercice provides absolutely no cardiovascular benefit and a rating of 10 is the most beneficial known exercise for the cardiovascular system.
      2. If the property is learning curve, a rating of 1 means the exercise requires no learning, while a rating of 10 means the exercise is the most difficult to learn and requires many repititions in order to learn and perform correctly.
      3. If the property is skill complexity, a rating of 1 requires no physical skill at all, while a rating of 10 means it requires the utmost physical skill to perform the exercise.
      4. If the property is mental engagement, a rating of 1 requires no mental engagement at all, while a rating of 10 means its the most mentally engaging exercise known.
      5. If the property is strength gain potential, a rating of 1 would yield no gains in human strength, while a rating of 10 means the exercise would yield the most physical strength gains compared to all exercises.

      How would you rate the following exercise in terms of {property_}: {noun}?
      '''

      prompt_val = _template.format(property_=property_val, noun=noun)
      messages = [
          {"role": "user", "content": prompt_val},
          ]
      response = pipe(messages)
      response_parsed = get_llm_response(response, prompt_response=True)
      if response_parsed:
        try:
          result = response_parsed
          MASTER[noun]["properties"].add(property_val)
          if property_val not in MASTER[noun]["metdata"]:
            MASTER[noun]["metdata"][property_val] = copy.deepcopy(_meta_shell)
            MASTER[noun]["metdata"][property_val]["values"].append(result)
          else:
            MASTER[noun]["metdata"][property_val]["values"].append(result)

        except Exception as e:
          print(e)
          continue


  with open("sample_data/ontology.pickle", "wb") as f:
    pickle.dump(MASTER, f)


In [ ]:
for i in range(5):
  print(i)
  run_student_387331()

0
1
2
3
4


In [ ]:
#"435777_"

import copy

def run_student_435777():
  _meta_shell = {'values': [], 'min': '', 'max': '', 'mean': '', 'std': ''}

  student = '435777_'

  nouns = student_object[student]['nouns']
  properties = student_object[student]['properties']
  for noun in nouns:
    for property_val in properties:
      if property_val == "innovation" or property_val == "endurance":
        continue
      _template = '''
      You are a semantic classifier of motorcycle models and are tasked to rate these motorcycle models in terms of a particular property in a range from 1-10 only.
      You must respond with a numeric rating ONLY in range from 1-10 based on the propery.  Only respond with a number and nothing else.
      1. If the property is speed, a rating of 1 means it is the slowest motorcycle on the market in terms of top speed in mph, while a rating of 10 means the motorcycle model has the fastest top speed in mph.
      2. If the property is handling, a rating of 1 means it has the worst manueverability of all motorcycle models while a rating of 10 means it is the most manueverable motorcycle of all known motorycle models.
      3. If the property is safety, a rating of 1 means it is the most unsafe motorcycle model based on probability of accident based on speed and handling, while a rating of 10 would be the most safe motorcycle model.
      4. If the property is popularity, a rating of 1 means it has the lowest sales compared to all models, while a rating of 10 means it is the most popular motorcycle in terms of sales.
      5. If the property is difficulty, a rating of 1 means it is the least difficult to drive, while a rating of 10 means it is the most difficult to drive.

      How would you rate the following motorcycle model in terms of {property_}: {noun}?
      '''


      prompt_val = _template.format(property_=property_val, noun=noun)
      messages = [
          {"role": "user", "content": prompt_val},
          ]
      response = pipe(messages)
      response_parsed = get_llm_response(response, prompt_response=True)
      if response_parsed:
        try:
          result = response_parsed
          MASTER[noun]["properties"].add(property_val)
          if property_val not in MASTER[noun]["metdata"]:
            MASTER[noun]["metdata"][property_val] = copy.deepcopy(_meta_shell)
            MASTER[noun]["metdata"][property_val]["values"].append(result)
          else:
            MASTER[noun]["metdata"][property_val]["values"].append(result)

        except Exception as e:
          print(e)
          continue


  with open("sample_data/ontology.pickle", "wb") as f:
    pickle.dump(MASTER, f)


In [ ]:
for i in range(5):
  print(i)
  run_student_435777()

0
1
2
3
4


In [ ]:
#395422_

import copy

def run_student_395422():
  _meta_shell = {'values': [], 'min': '', 'max': '', 'mean': '', 'std': ''}

  student = '395422_'

  nouns = student_object[student]['nouns']
  properties = student_object[student]['properties']
  for noun in nouns:
    for property_val in properties:
      if property_val == "height advantage":
        continue
      _template = '''
      You are a semantic classifier of famous NBA Basketball players and are tasked to rate these athletes in terms of a particular property in a range from 1-10 only.
      You must respond with a numeric rating ONLY in range from 1-10 based on the propery.  Only respond with a number and nothing else.
      1. If the property is scoring ability, a rating of 1 means that the basketball player's points per game average for a career was minimumal, while a rating of 10 would be the highest points per game average across all nba players.
      2. If the property is defense, a rating of 1 means the athlete was not known for shot blocking, or steals, and didnt receive any defensive awards, while a rating of 10 means the athlete had multiple defensive awards, and was known for shot blocking and steals.
      3. If the property is career endurance, a rating of 1 means the athlete played in minimal seasons of NBA such as 2 or less, while a rating of 10 would mean the athlete played in the highest number of seasons compared to all past and present NBA players.
      4. If the property is championship contribution, a rating of 1 means the athelete has 0 nba championship rings as a player, while a rating of 10 means that the athlete had 11 or more nba championships as a player.
      5. If the property is win contribution, a rating of 1 means means the athlete has less than 400 career wins, while a rating of 10 means the athlete has over 1100 career wins.

      How would you rate the following nba player in terms of {property_}: {noun}?
      '''


      prompt_val = _template.format(property_=property_val, noun=noun)
      messages = [
          {"role": "user", "content": prompt_val},
          ]
      response = pipe(messages)
      response_parsed = get_llm_response(response, prompt_response=True)

      if response_parsed:
        try:
          result = response_parsed
          MASTER[noun]["properties"].add(property_val)
          if property_val not in MASTER[noun]["metdata"]:
            MASTER[noun]["metdata"][property_val] = copy.deepcopy(_meta_shell)
            MASTER[noun]["metdata"][property_val]["values"].append(result)
          else:
            MASTER[noun]["metdata"][property_val]["values"].append(result)

        except Exception as e:
          print(e)
          continue


  with open("sample_data/ontology.pickle", "wb") as f:
    pickle.dump(MASTER, f)


In [ ]:
for i in range(5):
  print(i)
  run_student_395422()


0
1
2
3
4


In [ ]:
#432958
import copy

def run_student_432958():
  _meta_shell = {'values': [], 'min': '', 'max': '', 'mean': '', 'std': ''}

  student = '432958_'
  prompts = student_object[student]['prompts']
  nouns = student_object[student]['nouns']
  for noun in nouns:
    for prompt in prompts:
      prompt_val = prompt['prompt']
      property_val = prompt['property']
      messages = [
          {"role": "user", "content": prompt_val.format(noun)},
          ]
      response = pipe(messages)
      response_parsed = get_llm_response(response, prompt_response=True)
      if response_parsed:
        try:
          result = response_parsed
          MASTER[noun]["properties"].add(property_val)
          if property_val not in MASTER[noun]["metdata"]:
            MASTER[noun]["metdata"][property_val] = copy.deepcopy(_meta_shell)
            MASTER[noun]["metdata"][property_val]["values"].append(result)
          else:
            MASTER[noun]["metdata"][property_val]["values"].append(result)

        except Exception as e:
          print(e)
          continue


  with open("sample_data/ontology.pickle", "wb") as f:
    pickle.dump(MASTER, f)






In [ ]:
for i in range(5):
  run_student_432958()

edibility oak tree 1.0
longevity oak tree 10.0
woodenness oak tree 9.0
height oak tree 3.0
beauty oak tree 8.0
edibility maple tree 7.0
longevity maple tree 9.0
woodenness maple tree 8.0
height maple tree 9.0
beauty maple tree 8.0
edibility apple tree 10.0
longevity apple tree 7.0
woodenness apple tree 9.0
height apple tree 9.0
beauty apple tree 8.0
edibility cherry blossom tree 8.0
longevity cherry blossom tree 7.0
woodenness cherry blossom tree 3.0
height cherry blossom tree 9.0
beauty cherry blossom tree 9.0
edibility pine tree 1.0
longevity pine tree 9.0
woodenness pine tree 8.0
height pine tree 9.0
beauty pine tree 8.0
edibility cedar tree 2.0
longevity cedar tree 7.0
woodenness cedar tree None
height cedar tree 10.0
beauty cedar tree 8.0
edibility sequoia 1.0
longevity sequoia None
woodenness sequoia 10.0
height sequoia None
beauty sequoia 9.0
edibility olive tree 10.0
longevity olive tree 8.0
woodenness olive tree 7.0
height olive tree 10.0
beauty olive tree 8.0
edibility avocad

In [ ]:
#"344833_"

import copy

def run_student_344833():
  _meta_shell = {'values': [], 'min': '', 'max': '', 'mean': '', 'std': ''}

  student = '344833_'

  nouns = student_object[student]['nouns']
  properties = student_object[student]['properties']
  for noun in nouns:
    for property_val in properties:

      _template = '''
      You are a semantic classifier of various types of rocks and minerals and are tasked to rate these rocks/minerals in terms of a particular property in a range from 1-10 only.
      You must respond with a numeric rating ONLY in range from 1-10 based on the propery.  Only respond with a number and nothing else.

      How would you rate the following rock/mineral in terms of {property_}: {noun}?
      '''


      prompt_val = _template.format(property_=property_val, noun=noun)

      messages = [
          {"role": "user", "content": prompt_val},
          ]
      response = pipe(messages)
      response_parsed = get_llm_response(response, prompt_response=True)


      if response_parsed:
        try:
          result = response_parsed
          MASTER[noun]["properties"].add(property_val)
          if property_val not in MASTER[noun]["metdata"]:
            MASTER[noun]["metdata"][property_val] = copy.deepcopy(_meta_shell)
            MASTER[noun]["metdata"][property_val]["values"].append(result)
          else:
            MASTER[noun]["metdata"][property_val]["values"].append(result)

        except Exception as e:
          print(e)
          continue


  with open("sample_data/ontology.pickle", "wb") as f:
    pickle.dump(MASTER, f)


In [ ]:
for i in range(5):
  print(i)
  run_student_344833()

0
1
2
3
4


These are the remaining nouns that we were unable to use student data for.   

Loading a LLM generated categorization of the final 249 nouns to make the prompts more targeted

In [ ]:
import json


remaining_nouns_grouped = json.load(open("sample_data/grouped_nouns.json"))

In [ ]:
import copy

def run_remaining():
  _meta_shell = {'values': [], 'min': '', 'max': '', 'mean': '', 'std': ''}

  MAPPING =  {"People & Professions": ["education", "difficulty", "salary"],
            "Vehicles & Transportation": ["size", "speed", "weight", "fuel efficiency", "aerodynamic", "noise level"],
              "Tools & Equipment": ["size", "noise level", "weight"],
              "Places & Structures": ["size", "height", "length"],
              "Animals": ["size",
      "speed",
      "strength",
      "intelligence",
      "aggressiveness",
      "camouflage",
      "lifespan",
      "social_behavior",
      "agility",
      "danger_to_humans"],
              "Objects & Household Items": ["size", "noise level", "weight"]}

  for category, properties in MAPPING.items():
    print(category)
    for property_ in properties:
      for noun in remaining_nouns_grouped[category]:
        _template = '''You are a semantic classifier of various types of {category} and are tasked to rate these {category} in terms of a particular property in a range from 1-10 only.
        You must respond with a numeric rating ONLY in range from 1-10 based on the propery.  Only respond with a number and nothing else.

        How would you rate the following {category} in terms of {property_}: {noun}?
        '''
        prompt_val = _template.format(category=category, property_=property_, noun=noun)

        messages = [
            {"role": "user", "content": prompt_val},
            ]
        response = pipe(messages)
        response_parsed = get_llm_response(response, prompt_response=True)

        if response_parsed:
          try:
            result = response_parsed
            MASTER[noun]["properties"].add(property_)
            if property_ not in MASTER[noun]["metdata"]:
              MASTER[noun]["metdata"][property_] = copy.deepcopy(_meta_shell)
              MASTER[noun]["metdata"][property_]["values"].append(result)
            else:
              MASTER[noun]["metdata"][property_]["values"].append(result)

          except Exception as e:
            print(e)
          continue


  with open("sample_data/ontology.pickle", "wb") as f:
    pickle.dump(MASTER, f)


In [ ]:
for i in range(5):
  print("***********************", i, "*************************")
  run_remaining()

*********************** 0 *************************
People & Professions
Vehicles & Transportation
Tools & Equipment
Places & Structures
Animals
Objects & Household Items
*********************** 1 *************************
People & Professions
Vehicles & Transportation
Tools & Equipment
Places & Structures
Animals
Objects & Household Items
*********************** 2 *************************
People & Professions
Vehicles & Transportation
Tools & Equipment
Places & Structures
Animals
Objects & Household Items
*********************** 3 *************************
People & Professions
Vehicles & Transportation
Tools & Equipment
Places & Structures
Animals
Objects & Household Items
*********************** 4 *************************
People & Professions
Vehicles & Transportation
Tools & Equipment
Places & Structures
Animals
Objects & Household Items


In [ ]:
import copy
def final_run():
  _meta_shell = {'values': [], 'min': '', 'max': '', 'mean': '', 'std': ''}
  REMAINING_MAP = {'Weather': {'nouns': ['tornado'], 'properties': ["warm",
        "humid",
        "intense",
        "long",
        "frequent",
        "large",
        "visible",
        "impactful",
        "strong",
        "accurate"]},
        'Professions': {'nouns': ['soldier', 'singer', 'scientist'], 'properties': ["education", "difficulty", "salary"]},
        'Food Items': {'nouns': ['burger', 'salad', 'cookie', 'cake','bagel', 'cracker', 'soup', 'chocolate'], 'properties': ["weight",
        "size",
        "temperature",
        "sweetness",
        "savoriness",
        "healthiness"]}}

  for category, metadata in REMAINING_MAP.items():
    nouns = metadata['nouns']
    properties = metadata['properties']
    for noun in metadata['nouns']:
      for property_ in properties:
          _template = '''You are a semantic classifier of various types of {category} and are tasked to rate these {category} in terms of a particular property in a range from 1-10 only.
          You must respond with a numeric rating ONLY in range from 1-10 based on the propery.  Only respond with a number and nothing else.

          How would you rate the following {category} in terms of {property_}: {noun}?
          '''
          prompt_val = _template.format(category=category, property_=property_, noun=noun)

          messages = [
              {"role": "user", "content": prompt_val},
              ]
          response = pipe(messages)
          response_parsed = get_llm_response(response, prompt_response=True)
          print(property_, noun, response_parsed)
          if response_parsed:
            try:
              result = response_parsed
              MASTER[noun]["properties"].add(property_)
              if property_ not in MASTER[noun]["metdata"]:
                MASTER[noun]["metdata"][property_] = copy.deepcopy(_meta_shell)
                MASTER[noun]["metdata"][property_]["values"].append(result)
              else:
                MASTER[noun]["metdata"][property_]["values"].append(result)

            except Exception as e:
              print(e)
            continue


    with open("sample_data/ontology.pickle", "wb") as f:
      pickle.dump(MASTER, f)


In [ ]:
for i in range(5):
  print(i)
  final_run()

0
warm tornado 1.0
humid tornado 1.0
intense tornado 10.0
long tornado 1.0
frequent tornado 2.0
large tornado 10.0
visible tornado 1.0
impactful tornado 10.0
strong tornado 10.0
accurate tornado 3.0
education soldier 5.0
difficulty soldier 8.0
salary soldier 4.0
education singer 7.0
difficulty singer 7.0
salary singer 5.0
education scientist 8.0
difficulty scientist 7.0
salary scientist 7.0
weight burger 8.0
size burger 7.0
temperature burger 7.0
sweetness burger 3.0
savoriness burger 8.0
healthiness burger 3.0
weight salad 3.0
size salad 3.0
temperature salad 7.0
sweetness salad 1.0
savoriness salad 2.0
healthiness salad 8.0
weight cookie 2.0
size cookie 7.0
temperature cookie 7.0
sweetness cookie 7.0
savoriness cookie 3.0
healthiness cookie 4.0
weight cake 8.0
size cake 9.0
temperature cake 7.0
sweetness cake 8.0
savoriness cake 2.0
healthiness cake 2.0
weight bagel 4.0
size bagel 5.0
temperature bagel 7.0
sweetness bagel 2.0
savoriness bagel 7.0
healthiness bagel 3.0
weight cracker 